In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):
    def __init__(self,img_dimensions):
        super().__init__()
        self.disc=nn.Sequential(
            nn.Linear(img_dimensions,128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.disc(x)


In [3]:
class Generator(nn.Module):
    def __init__(self,z_dim,img_dim):
        super().__init__()
        self.gen=nn.Sequential(
            nn.Linear(z_dim,256),
            nn.LeakyReLU(0.1),
            nn.Linear(256,img_dim),
            nn.Tanh(),

        )

    def forward(self,x):
        return self.gen(x)


In [4]:
#Hyperparameters etc.
"""GANS are very sensitive to these hyperparameters"""

device="cuda" if torch.cuda.is_available() else "cpu"
lr=1e-4
z_dim=64 
img_dims=28*28*1
batch_size=32
num_epochs=50

In [5]:
disc=Discriminator(img_dims).to(device)
gen=Generator(z_dim,img_dims).to(device)
fixed_noise= torch.randn((batch_size, z_dim)).to(device)
transforms=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.50,),(0.50,))
    ]
)


In [7]:
dataset=datasets.MNIST(root="dataset/",transform=transforms,download=True)
loader=DataLoader(dataset,batch_size,shuffle=True)
opt_disc=optim.Adam(disc.parameters(),lr=lr)
opt_gen=optim.Adam(gen.parameters(),lr=lr)
criterion = nn.BCELoss()
writer_fake=SummaryWriter(f"runs2/GAN_MNIST/fake")
writer_real=SummaryWriter(f"runs2/GAN_MNIST/real")
step=0


In [12]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1,784).to(device)
        batch_size=real.shape[0]

        #Train Discriminator: max log(D(real))+log(1-D(G(z)))

        noise= torch.randn(batch_size,z_dim).to(device)
        fake=gen(noise)
        disc_real= disc(real).view(-1)
        lossD_real=nn.BCELoss()(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake=criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_fake+lossD_real) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        #Train Generator min log(1-D(G(Z))) <-> max log(D(G(z)))

        output=disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1
                


Generator(
  (gen): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Tanh()
  )
)
Epoch [0/50] Batch 0/1875                       Loss D: 0.4530, loss G: 0.9400
Generator(
  (gen): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Tanh()
  )
)
Generator(
  (gen): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Tanh()
  )
)
Generator(
  (gen): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Tanh()
  )
)
Generator(
  (gen): Sequential(
    (0): Linear(in_featur

KeyboardInterrupt: 